In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import sys
import asyncio

# Fix for Windows issues in Jupyter notebooks
if sys.platform == "win32":
    # 1. Use ProactorEventLoop for subprocess support
    if not isinstance(asyncio.get_event_loop_policy(), asyncio.WindowsProactorEventLoopPolicy):
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
    # 2. Redirect stderr to avoid fileno() error when launching MCP servers
    if "ipykernel" in sys.modules:
        sys.stderr = sys.__stderr__


## Local MCP server
---

More documentation on transports, here: https://modelcontextprotocol.io/specification/2025-11-25/basic/transports

In [3]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "local_server": {
                "transport": "stdio", # can be stdio or Streaming HTTP depending on the server
                "command": "python",
                "args": ["resources/2.1_mcp_server.py"],
            }
    }
)

In [4]:
# get tools
tools = await client.get_tools()

# get resources
resources = await client.get_resources("local_server")

# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [5]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=tools,
    system_prompt=prompt
)

In [6]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config
)

In [7]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Tell me about the langchain-mcp-adapters library', additional_kwargs={}, response_metadata={}, id='115b6233-4c98-445e-b6b9-3f3c827f855f'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 348, 'prompt_tokens': 271, 'total_tokens': 619, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D8io8cXJrnvozATGdjgjIxgLVaIAA', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c561c-860c-7e83-bf69-0cf9279d6a3b-0', tool_calls=[{'name': 'search_web', 'args': {'query': 'langchain-mcp-adapters'}, 'id': 'call_GdiFjRlbXmCXJa9pSptQKkSM', 'type': 'tool_call'}], invalid_tool_calls=[], us

## Online MCP
---

Time MCP server details here: https://mcp.so/server/time/modelcontextprotocol

In [8]:
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/New_York"
            ]
        }
    }
)

tools = await client.get_tools()

In [9]:
agent = create_agent(
    model="gpt-5-nano",
    tools=tools,
)

In [10]:
question = HumanMessage(content="What time is it?")

response = await agent.ainvoke(
    {"messages": [question]}
)

pprint(response)

{'messages': [HumanMessage(content='What time is it?', additional_kwargs={}, response_metadata={}, id='36ba17a5-998a-4ca5-a65b-3ee51ec8fc81'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 347, 'prompt_tokens': 296, 'total_tokens': 643, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D8iotvXwfxCnj4xHXcybEY9Yh9z6o', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c561d-3e17-7643-b5c2-84d3a9dfdf33-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'America/New_York'}, 'id': 'call_qUijeNrKNblpbRFqt2j5FBPn', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens':

In [11]:
question = HumanMessage(content="What time is it in Kolkata?")

response = await agent.ainvoke(
    {"messages": [question]}
)

pprint(response)

{'messages': [HumanMessage(content='What time is it in Kolkata?', additional_kwargs={}, response_metadata={}, id='dedfc1ee-6694-4088-b93b-050dcbada13f'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 219, 'prompt_tokens': 298, 'total_tokens': 517, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D8ipFH4HwAvgUKi3eepQI1QNncP6y', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c561d-934f-7161-a012-97e3013a5c3b-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Kolkata'}, 'id': 'call_mIbPpkM0dH4Z5LVdrTpxWUFb', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_t

---
Kiwi Travel MCP Server Example: https://mcp.so/server/kiwi-travel-mcp/Vytautas%20Dargis

In [18]:
client = MultiServerMCPClient(
    {
    "travel_server": {
        "transport": "streamable_http",
        "url": "https://mcp.kiwi.com"
    }
  }
    )

tools = await client.get_tools()

In [19]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model="gpt-5-nano",
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt="You are a travel agent. No follow up questions."
)

In [20]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}} # Separate thread for this agent

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="What direct flights are available from Kolkata to Singapore in March 2026?")]},
    config=config
)

In [21]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What direct flights are available from Kolkata to Singapore in March 2026?', additional_kwargs={}, response_metadata={}, id='3ae71a8c-d40a-410b-8755-b44d628e18b6'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1555, 'prompt_tokens': 1229, 'total_tokens': 2784, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1216, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D8j0x0tke3anTE0V4ieQe5XeV1GtZ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c5628-a782-7b61-a4a0-61f9c606b9d1-0', tool_calls=[{'name': 'search-flight', 'args': {'flyFrom': 'Kolkata', 'flyTo': 'Singapore', 'departureDate': '01/03/2026', 'departureDateF

In [22]:
print(response["messages"][-1].content)

Direct flights from Kolkata (CCU) to Singapore (SIN) in March 2026:

| Route | Schedule (local) | Cabin | Price (EUR) | Booking link |
|---|---|---|---|---|
| Kolkata CCU → Singapore SIN | 01/03 02:00 → 01/03 08:55 (4h 25m) | Economy | 147 EUR | https://on.kiwi.com/YaPN2J |

Direct flight notes:
- This is the only non-stop option found for March 2026 in the results.

Wishing you a wonderful trip! Fun fact: Singapore is known for its iconic Gardens by the Bay and its iconic skyline lit up at night.
